In [ ]:
import pandas as pd
import numpy as np
import panel as pn
import hvplot.pandas

df = pd.DataFrame({"x": np.random.randint(0, 10, 10), "Date": pd.date_range(end=pd.Timestamp.now(), periods=10)})

def update():
    global df
    print("hel")


    # df_new = pd.concat([df, new_data], axis=0)
    # df_new = pd.DataFrame(df, new_data)
    # new_row = {'x': np.random.randint(0, 10, 3), 'Date':pd.date_range(end=pd.Timestamp.now(), periods=3)}
    # df = df.append(new_row, ignore_index=True)

    # new_row = np.random.randint(0, 10, 3),pd.date_range(end=pd.Timestamp.now(), periods=3)
    # df.loc[len(df.index)] =new_row

    # USE CONCAT
    # display(df)
    new_row = pd.DataFrame({'x': np.random.randint(0, 10, 3), 'Date':pd.date_range(end=pd.Timestamp.now(), periods=3)})
    # display(new_row)
    df = pd.concat([df, new_row],  ignore_index = True, axis=0)
    # display(df)
    df.reset_index()

    # display(df)

    print(f"df {len(df)}")
    return df

def panel_app():
    global df
    plot = df.hvplot.line(x="x", y="Date", datashade=True)
    print("in panel_app")
    cb = pn.state.add_periodic_callback(lambda: plot.data.update(update), 200, timeout=5000)


    toggle = pn.widgets.Toggle(name='Toggle callback', value=True)


    toggle.link(cb, bidirectional=True, value='running')

    return pn.Column(plot, toggle)

pn.serve(panel_app)

In [ ]:
import pandas as pd
import numpy as np
import panel as pn
import hvplot.pandas
import param

df = pd.DataFrame({"x": np.random.randint(0, 10, 10), "y": np.random.randint(0, 10, 10)})

def update():
    global df
    try:
        new_row = pd.DataFrame({"x": np.random.randint(0, 10, 3), "y": np.random.randint(0, 10, 3)})
        df_new = pd.concat([df, new_row], ignore_index=True, axis=0)
        df = df_new
        print(len(df))
        return df
    except Exception as e:
        print(f"An error occurred: {e}")

@param.depends('df')
def plot_func():
    return df.hvplot.scatter(x="x", y="y")

def panel_app():
    global df
    # df = pd.DataFrame({"x": np.random.randint(0, 10, 10), "y": np.random.randint(0, 10, 10)})
    plot = plot_func()
    cb = pn.state.add_periodic_callback(lambda: plot.data.update(update()), 200, timeout=1000) #
    toggle = pn.widgets.Toggle(name='Toggle callback', value=True)
    toggle.link(cb, bidirectional=True, value='running')
    return pn.Column(plot, toggle)

pn.serve(panel_app)

In [ ]:
import pandas as pd
import numpy as np
import panel as pn
import hvplot.pandas
import holoviews as hv
import time

def generate_data():
    while True:
        yield pd.DataFrame({"x": np.random.randint(0, 10, 10), "y": np.random.randint(0, 10, 10)})
        time.sleep(1)

def update(data):
    return hv.Points(data, kdims=["x"], vdims=["y"])

def panel_app():
    data_stream = hv.streams.Stream(source=generate_data())
    plot = hv.DynamicMap(update, streams=[data_stream])
    plot = plot.opts(width=400, height=400, xlim=(0, 10), ylim=(0, 10))
    return plot

pn.serve(panel_app)

In [ ]:
import pandas as pd
import holoviews as hv
import hvplot.pandas
from holoviews.streams import Pipe, Buffer
hv.extension('bokeh')

# Create a dataframe
df = pd.DataFrame({'x': [1, 2, 3, 4, 5], 'y': [2, 4, 6, 8, 10]})

# Define a function that converts the dataframe to a holoviews object
def df_to_hv(df):
    return hv.Points(df, kdims=['x'], vdims=['y'])

# Create a Pipe object that will receive the dataframe updates
pipe = Pipe(data=df)

# Create a Buffer object that will hold the updated holoviews object
buffer = Buffer(df_to_hv(df))

# Define a callback function that updates the Buffer object whenever the Pipe object receives new data
def callback(data):
    buffer.send(df_to_hv(data))

# Attach the callback function to the Pipe object
pipe.add_subscriber(callback)

# Plot the dataframe using hvplot
plot = df.hvplot.scatter(x='x', y='y')

# Combine the plot and the Buffer object
layout = plot * buffer

# Display the layout
layout

In [ ]:
import time
import hvplot.pandas
import pandas as pd

# Define a function that updates a dictionary with a stream of data
def update_dict(data):
    # Initialize the dataframe
    df = pd.DataFrame(columns=['x', 'y'])

    # Loop over the data and update the dataframe
    for x, y in data:
        df = df._append({'x': x, 'y': y}, ignore_index=True)

        # Plot the updated dataframe using hvplot
        plot = df.hvplot.scatter(x='x', y='y', xlim=(0, 6), ylim=(0, 12))

        # Display the plot
        display(plot)

        # Pause for a short time to simulate real-time streaming
        time.sleep(1)

# Define some example data
data = [(1, 2), (2, 4), (3, 6), (4, 8), (5, 10)]

# Update the dictionary with the data stream
update_dict(data)

In [ ]:
import time
import numpy as np
import pandas as pd
import holoviews as hv
import streamz
import streamz.dataframe

from holoviews import opts
from holoviews.streams import Pipe, Buffer

hv.extension('bokeh')

In [ ]:
pipe = Pipe(data=[])
vector_dmap = hv.DynamicMap(hv.VectorField, streams=[pipe])
vector_dmap.opts(color='Magnitude', xlim=(-1, 1), ylim=(-1, 1))

In [ ]:
x,y  = np.mgrid[-10:11,-10:11] * 0.1
sine_rings  = np.sin(x**2+y**2)*np.pi+np.pi
exp_falloff = 1/np.exp((x**2+y**2)/8)

for i in np.linspace(0, 1, 25):
    time.sleep(0.1)
    pipe.send((x,y,sine_rings*i, exp_falloff))

In [ ]:
example = pd.DataFrame({'x': [], 'y': [], 'count': []}, columns=['x', 'y', 'count'])
dfstream = Buffer(example, length=100, index=False)
curve_dmap = hv.DynamicMap(hv.Curve, streams=[dfstream])
point_dmap = hv.DynamicMap(hv.Points, streams=[dfstream])

In [ ]:
(curve_dmap * point_dmap).opts(
    opts.Points(color='count', line_color='black', size=5, padding=0.1, xaxis=None, yaxis=None),
    opts.Curve(line_width=1, color='black'))

In [ ]:
def gen_brownian():
    x, y, count = 0, 0, 0
    while True:
        x += np.random.randn()
        y += np.random.randn()
        count += 1
        yield pd.DataFrame([(x, y, count)], columns=['x', 'y', 'count'])

brownian = gen_brownian()
for i in range(200):
    dfstream.send(next(brownian))

In [ ]:
simple_sdf = streamz.dataframe.Random(freq='10ms', interval='100ms')
print(simple_sdf.index)
simple_sdf.example.dtypes

In [ ]:
sdf = (simple_sdf-0.5).cumsum()
hv.DynamicMap(hv.Curve, streams=[Buffer(sdf.x)]).opts(width=500, show_grid=True)


In [ ]:

import pandas as pd
from time import sleep
from datetime import datetime

from streamz import Stream
from streamz.dataframe import DataFrame


#Set up a dummy dataframe
ddf=pd.DataFrame({'a':range(1,5),'b':range(11,15)})
#Create a stream source
source = Stream()

#Create a dataframe model for the stream
samples = pd.DataFrame({'a':[0],'b':[0]})

#Creating the streaming dataframe
sdf = DataFrame(source, example=samples)

#That window thing again...
df = sdf.window(4).full()

#FUnction to run when an item appears on the stream
def stest(r):
    print(datetime.now())
    print(r)

#Add the callback function to streamer
df.stream.sink(stest)

#This does stream from the dataframe
for i in range(len(ddf)):
    source.emit(ddf.iloc[i])
    sleep(2)

df

In [ ]:
from tornado import gen
from tornado.ioloop import PeriodicCallback

from holoviews.streams import Buffer

import holoviews as hv
hv.extension('bokeh')


import numpy as np
import pandas as pd

df = pd.DataFrame({'x':range(1000), 'y':np.sin(range(1000))})

rowcount = 0
maxrows = 1000

dfbuffer = Buffer(np.zeros((0, 2)), length=20)

@gen.coroutine
def g():
    global rowcount
    item = df[['x','y']].iloc[rowcount].values.tolist()
    dfbuffer.send(np.array([item]))
    rowcount += 1

    if rowcount>=maxrows:
        cbdf.stop()



#How can we get the thing to stop?

cbdf = PeriodicCallback(g, 500)
cbdf.start()
hv.DynamicMap(hv.Curve, streams=[dfbuffer]).opts(padding=0.1, width=600, color = 'green',)

In [ ]:
simple_sdf.start()

In [ ]:
! pip install "notebook>=5.3" "ipywidgets>=7.5"

In [ ]:
# ! pip install --upgrade ipywidgets==7.7.2
! pip install --upgrade ipywidgets==7.6 

In [ ]:
! jupyter --v

In [ ]:
from streamz import Stream

def query(df):
    return df[df.name == 'Alice']

def aggregate(acc, df):
    return acc + df.amount.sum()

stream = Stream()
stream.map(query).accumulate(aggregate, start=0)

In [ ]:
import pandas as pd
from streamz.dataframe import DataFrame
import panel as pn

example = pd.DataFrame({'name': [], 'amount': []})
sdf = DataFrame(stream, example=example)

pn.Row(sdf[sdf.name == 'Alice'].amount.sum()).show()

In [ ]:
import time
import numpy as np
import pandas as pd
import holoviews as hv
import streamz
import streamz.dataframe

from holoviews import opts
from holoviews.streams import Pipe, Buffer

hv.extension('bokeh')

In [ ]:
point_source = streamz.Stream()
pipe = Pipe(data=pd.DataFrame({'x': [1,2,3], 'y': [1,2,3], 'count': 3}))
point_source.sliding_window(20).map(pd.concat).sink(pipe.send) # Connect streamz to the Pipe
scatter_dmap = hv.DynamicMap(hv.Scatter, streams=[pipe])

In [ ]:
pn.serve(pn.Row(scatter_dmap.opts(bgcolor='black', color='count', ylim=(-4, 4), show_legend=False)))

In [ ]:
for i in range(50):
    df = pd.DataFrame({'x': np.random.rand(3), 'y': np.random.randn(3), 'count': i},
                      columns=['x', 'y', 'count'])
    point_source.emit(df) # emit pushes data at this point into the stream that is point_source
    # time.sleep(1)

In [ ]:
def gen_brownian():
    x, y, count = 0, 0, 0
    while True:
        x += np.random.randn()
        y += np.random.randn()
        count += 1
        yield pd.DataFrame([(x, y, count)], columns=['x', 'y', 'count'])

brownian = gen_brownian()
for i in range(200):
    dfstream.send(next(brownian))

### stream points

In [ ]:
simple_sdf = streamz.dataframe.Random(freq='10ms', interval='100ms')
# print(simple_sdf.index)
# simple_sdf.example.dtypes

sdf = simple_sdf
# pn.serve(pn.Row(hv.DynamicMap(hv.Curve, streams=[Buffer(sdf.x)]).opts(width=500, show_grid=True)))

In [ ]:
pn.serve(pn.Row(hv.DynamicMap(hv.Scatter, streams=[Buffer(sdf.x)]).redim.label(x='value', index='time')))


# TRANSLATE INTO WITH DATES

In [ ]:
## MAL
simple_sdf = streamz.dataframe.Random(freq='10ms', interval='100ms')
print(simple_sdf.index)
simple_sdf.example.dtypes

sdf = (simple_sdf-0.5).cumsum()
pn.serve(pn.Row(hv.DynamicMap(hv.Curve, streams=[Buffer(sdf.x)]).opts(width=500, show_grid=True)))

In [ ]:
sdf = streamz.dataframe.Random(freq='10ms', interval='100ms')
print(simple_sdf.index)
simple_sdf.example.dtypes

sdf = (simple_sdf-0.5).cumsum()
pn.serve(pn.Row(hv.DynamicMap(hv.Curve, streams=[Buffer(sdf.x)]).opts(width=500, show_grid=True)))

In [ ]:
sdf = (simple_sdf-0.5).cumsum()
pn.serve(pn.Row(hv.DynamicMap(hv.Curve, streams=[Buffer(sdf.x)]).opts(width=500, show_grid=True)))

In [ ]:
sdf.stop()

In [ ]:
!jupyter --version


In [ ]:
!pip install jupyterlab==1



In [ ]:

pn.param.ParamMethod.loading_indicator = True

# Defining the paramterised class 
class CustomDashboard(param.Parameterized):
    

    df = pd.DataFrame({"x": np.random.randint(0, 10, 10), "y": np.random.randint(0, 10, 10)})




    # @param.depends('show_all_dates', watch=True)
    # def toggle_on_change(self) :
    #     self.plot()
    
    



    
    #Main function for plotting. Too long, should be refactored with more functions
    @param.depends('df') # TODO why does this function not depend on 'wellbore'? is it bc this funciton is indrectly triggered as sensor updates based on wellbore which will trigger this function?
    def plot(self):
        return df.hvplot.scatter(x="x", y="y")


    
    #Run update sensor method to populate sensor dropdown
    def __init__(self, **params):
        super().__init__(**params)
        # self.update_sensors()









        


    def panel(self):
        return pn.Row(
            pn.Column(pn.Param(self.param), width = 300), pn.Row(self.plot)

        )

# Instantiate the dashboard
dashboard = CustomDashboard()

# Trigger initial sensor update
#dashboard.update_sensors()

# Display the dashboard
dashboard.panel().show()





In [ ]:
import hvplot.pandas  # noqa
import pandas as pd
import panel as pn

pn.extension()


def function_to_generate_dataframe(x):
    # x only used to trigger the function
    df = pd._testing.makeDataFrame()
    df["C"] = df["C"].round()
    return df


widget = pn.widgets.IntSlider(end=100)
hvplot.bind(function_to_generate_dataframe, widget).interactive().hvplot.scatter(x="A", y="B", by="C")

In [ ]:
!jupyter nbextension enable --py widgetsnbextension
